In [1]:
import os
import openai
import requests
import json
import sseclient

In [2]:
# openai.api_key = os.getenv("OPENAI_API_KEY")
# openai.Completion.create(
#   model="text-davinci-003",
#   prompt="Say this is a test",
#   max_tokens=7,
#   temperature=0
# )

In [3]:
# def send_request(prompt):
#     openai.api_key = os.getenv("OPENAI_API_KEY")
#     response = openai.Completion.create(
#       model="text-davinci-003",
#       prompt=prompt,
#       temperature=0
#     )
    
#     return response

In [4]:
# response = send_request("Say this is a test")
# response

In [5]:
# response.choices

In [6]:
real_prompt = """
You are an enthusiastic iTwinUI representative who loves helping users. iTwinUI is a UI library with a React components like `Button`, `Table`, `Header`, `Text`, etc. where all components follow the strict UX guidelines set by iTwinUI. Given the "Most relevant chunks" from the iTwinUI documentation website, give an answer in markdown (code and hierarchical bullet points, whenever possible) to the "User query". You should not use any information outside of "Most relevant chunks". If you are not confident about your answer, say "Sorry, I do not have a confident answer". NEVER guess an answer or inject your knowledge.

"User query":
Where should I place an Alert component? Like from a UX perspective 

"Most relevant chunks":
```json
[
  {
    "header": ["Alert", "Placement"],
    "content": "The alert notification can display in two different ways:"
  },
  {
    "header": ["Alert", "Placement", "Inline"],
    "content": "An inline alert appears embedded within the copy of the site as a block element. This means that when the user scrolls, the alert scrolls as well.\n\nInline alerts can optionally be dismissible. Once the alert is dismissed, the content below it will shift upwards due to the alert no longer consuming space."
  },
  {
    "header": ["Alert", "Placement", "Sticky"],
    "content": "A sticky alert appears full width stuck at the top of the window frame. That means if the user scrolls, the alert stays at the top of the frame and as the content scrolls underneath it. A sticky alert must always be dismissible. The alert appears above all other UI elements except for toast notifications and modals.\n\nIt is up to the development team wether or not the sticky alert should push the rest of the content down or not."
  },
]
```
""".strip()

real_prompt

'You are a enthusiastic iTwinUI representative who loves helping users. iTwinUI is a UI library with a React components like `Button`, `Table`, `Header`, `Text`, etc. where all components follow the strict UX guidelines set by iTwinUI. Given the "Most relevant chunks" from the iTwinUI documentation website, give an answer in markdown (code and hierarchical bullet points, whenever possible) to the "User query". You should not use any information outside of "Most relevant chunks". If you are not confident about your answer, say "Sorry, I do not have a confident answer". NEVER guess an answer or inject your knowledge.\n\n"User query":\nWhere should I place an Alert component? Like from a UX perspective \n\n"Most relevant chunks":\n```json\n[\n  {\n    "header": ["Alert", "Placement"],\n    "content": "The alert notification can display in two different ways:"\n  },\n  {\n    "header": ["Alert", "Placement", "Inline"],\n    "content": "An inline alert appears embedded within the copy of th

In [7]:
# real_prompt_response = send_request(real_prompt)

In [29]:
def generate_streaming_text(prompt, model="text-ada-001", max_tokens=30):
    headers = {
        "Content-Type": "application/json",
        "Accept": "text/event-stream",
        "Authorization": f"Bearer {openai.api_key}"
    }
    data = {
        "prompt": prompt,
        "model": model,
        "max_tokens": max_tokens,
        "temperature": 0.0,
        "stream":True
    }
    response = requests.post("https://api.openai.com/v1/completions", headers=headers, data=json.dumps(data))
    client = sseclient.SSEClient(response)
    for event in client.events():
        # print("HERE")
        if event.data != "[DONE]":
            response_json = json.loads(event.data)
            # print(response_json["choices"][0]["text"], end="", flush=True)

            if "choices" in response_json:
                # print(response_json["choices"][0]["text"], end="", flush=True)
                yield response_json["choices"][0]["text"]

        # if event.event == "data":
        #     response_json = json.loads(event.data)
        #     if "choices" in response_json:
        #         return response_json["choices"][0]["text"]
        #     else:
        #         return response_json
        
    # response_json = response.json()
    # return response_json

In [18]:
for chunk in generate_streaming_text(real_prompt, max_tokens=200):
    print(chunk, end="", flush=True)



The alert notification can display in two different ways:

-Sticky alerts: The alert appears full width stuck at the top of the window frame and as the content scrolls underneath it.
-Ongoing alerts: The alert appears embedded within the site as a block element and when the user scrolls, the alert scrolls as well.

-Sticky alerts should be:

-Dismissible: The alert appears full width stuck at the top of the window frame and as the content scrolls underneath it. When the user scrolls, the alert dismisses itself.

-Sticky alerts should be:

-Dismissible: The alert appears full width stuck at the top of the window frame and as the content scrolls underneath it. When the user scrolls, the alert dismisses itself.

In [19]:
for chunk in generate_streaming_text(real_prompt, model="text-babbage-001", max_tokens=300):
    print(chunk, end="", flush=True)



The most relevant chunks for this user query are:

-Header: "Alert", "Placement", "Inline"
-Content: "An inline alert appears embedded within the copy of the site as a block element. This means that when the user scrolls, the alert scrolls as well.
-Sticky: "A sticky alert appears full width stuck at the top of the window frame. That means if the user scrolls, the alert stays at the top of the frame and as the content scrolls underneath it. A sticky alert must always be dismissible. The alert appears above all other UI elements except for toast notifications and modals."

In [20]:
for chunk in generate_streaming_text(real_prompt, model="text-curie-001", max_tokens=300):
    print(chunk, end="", flush=True)



The Alert component can display in two different ways: as an inline alert or as a sticky alert. A sticky alert appears full width stuck at the top of the window frame. That means if the user scrolls, the alert stays at the top of the frame and as the content scrolls underneath it. A sticky alert must always be dismissible.

In [30]:
for chunk in generate_streaming_text(real_prompt, model="text-davinci-003", max_tokens=300):
    print(chunk, end="", flush=True)



It depends on the type of alert you are displaying. 

For an inline alert, it should appear embedded within the copy of the site as a block element. This means that when the user scrolls, the alert scrolls as well.

For a sticky alert, it should appear full width stuck at the top of the window frame. That means if the user scrolls, the alert stays at the top of the frame and as the content scrolls underneath it. A sticky alert must always be dismissible. The alert appears above all other UI elements except for toast notifications and modals. It is up to the development team wether or not the sticky alert should push the rest of the content down or not.

In [31]:
for chunk in generate_streaming_text(real_prompt, model="text-davinci-003", max_tokens=600):
    print(chunk, end="", flush=True)



It depends on the type of alert you are displaying. 

For an inline alert, it should appear embedded within the copy of the site as a block element. This means that when the user scrolls, the alert scrolls as well.

For a sticky alert, it should appear full width stuck at the top of the window frame. That means if the user scrolls, the alert stays at the top of the frame and as the content scrolls underneath it. A sticky alert must always be dismissible. The alert appears above all other UI elements except for toast notifications and modals. It is up to the development team wether or not the sticky alert should push the rest of the content down or not.

In [32]:
real_prompt_1 = """
You are an enthusiastic iTwinUI representative who loves helping users. iTwinUI is a UI library with a React components like `Button`, `Table`, `Header`, `Text`, etc. where all components follow the strict UX guidelines set by iTwinUI. Given the "Most relevant chunks" from the iTwinUI documentation website, give an answer in markdown to the "User query". You should not use any information outside of "Most relevant chunks". If you are not confident about your answer, say "Sorry, I do not have a confident answer". NEVER guess an answer or inject your knowledge.

"User query":
Where should I place an Alert component? Like from a UX perspective 

"Most relevant chunks":
```json
[
  {
    "header": ["Alert", "Placement"],
    "content": "The alert notification can display in two different ways:"
  },
  {
    "header": ["Alert", "Placement", "Inline"],
    "content": "An inline alert appears embedded within the copy of the site as a block element. This means that when the user scrolls, the alert scrolls as well.\n\nInline alerts can optionally be dismissible. Once the alert is dismissed, the content below it will shift upwards due to the alert no longer consuming space."
  },
  {
    "header": ["Alert", "Placement", "Sticky"],
    "content": "A sticky alert appears full width stuck at the top of the window frame. That means if the user scrolls, the alert stays at the top of the frame and as the content scrolls underneath it. A sticky alert must always be dismissible. The alert appears above all other UI elements except for toast notifications and modals.\n\nIt is up to the development team wether or not the sticky alert should push the rest of the content down or not."
  },
]

Your answer in markdown (hierarchical bullet points, whenever possible):
```
""".strip()

In [34]:
for chunk in generate_streaming_text(real_prompt_1, model="text-davinci-003", max_tokens=600):
    print(chunk, end="", flush=True)


The placement of an Alert component depends on the context of the page:

* Inline: 
  * Appears embedded within the copy of the site as a block element
  * Optionally be dismissible
  * Content below it will shift upwards once the alert is dismissed

* Sticky: 
  * Appears full width stuck at the top of the window frame
  * Stays at the top of the frame when the user scrolls
  * Must always be dismissible
  * Appears above all other UI elements except for toast notifications and modals
  * It is up to the development team wether or not the sticky alert should push the rest of the content down or not